# Chapter 11: *Off-policy Methods with Approximation

- Recall that in off-policy learning we seek to learn a vlaue function for a *target policy* $\pi$, given data due to a different *behavior policy b*
- In the prediction case, both policies are static and given, and we seek to learn either state values $\hat{v} \approx v_{\pi}$ or action values $\hat{q} \approx q_{\pi}$ 
- In the control case, action values are learned and both policies typically change during learning - $\pi$ being the greedy policy with respect to $\hat{q}$, and *b* being something more exploratory such as the eps-greedy policy w.r.t. $\hat{q}$
- The first challenge of off-policy learning has to do with the target of the update but we can deal with that by using importance sampling
- The second challenge of off-policy learning with function approximation is that the distribution of updates in the off-policy case is not according to the on-policy distribution
    - There are two general approaches, one that uses IS and one that develops true gradient methods... but it is not clear which of these approaches is most effective in practice

# 11.1 Semi-gradient Methods

- To convert tabular off-policy algorithms to semi-gradient form, we repalce the update to an array (V or Q) to an update to a weight vector(**w**), using the approximate vlaue function($\hat{v}$ or $\hat{q}$ and its gradient
- Many of these algorithms use the per-step importance sampling ratio:
$$\rho_{t} \doteq \rho_{t:t} = \frac{\pi(A_{t}|S_{t}}{b(A_{t}|S_{t})}$$
- The one-step, state-value algorithm is semi-gradient  off-policy TD(0) is:
$$ \textbf{w}_{t+1} \doteq \textbf{w}_{t} + \alpha\rho_{t}\delta_{t}\nabla\hat{v}(S_{t}, \textbf{w}_{t})$$
    - where $\gamma_{t}$ is defined depending on whether the task is episodic and discounted, or continuing and undiscounted using average reward:
        $$\gamma_{t} \doteq R_{t+1} + \gamma\hat{v}(S_{t+1}, \textbf{w}_{T}) - \hat{v}(S_{t},\textbf{w}_{t})$$ or $$\gamma_{t} \doteq R_{t+1} - \bar{R}_{t} + \hat{v}(S_{t+1}, \textbf{w}_{t}) - \hat{v}(S_{t}, \textbf{w}_{t})$$
- For action values, the one-step algorithm is semi-gradient Expected Sarsa :
$$ \textbf{w}_{t+1} \doteq \textbf{w}_{t} + \alpha\delta_{t}\nabla\hat{q}(S_{t}, A_{t} \textbf{w}_{t})$$ with episodic, then coninuing cases for delta:
$$ \delta_{t} \doteq R_{t+1} + \gamma \sum_{a} \pi(a|S_{t+1})\hat{q}(S_{t+1},a,\textbf{w}_{t}) - \hat{q}(S_{t}, A_{t}, \textbf{w}_{t})$$ or

$$ \delta_{t} \doteq R_{t+1} - \bar{R}_{t} + \sum_{a} \pi(a|S_{t+1})\hat{q}(S_{t+1},a,\textbf{w}_{t}) - \hat{q}(S_{t}, A_{t}, \textbf{w}_{t})$$

- Note theat this alforithm does not use importance sampling 

# 11.2 Examples of Off-policy Divergence

- The second part of the challenge of off-policy learning with function approximation is that the distribution of the updates does not match the on-policy distribution
- An issue that can happen is that one transition occurs repeatedly without one state being updated on other transitions.  This is possible under off-policy training because the behavior policy migt seles actions on those other transitions which the target policy never would.  Ro these transitions, $\rho_{t}$ whould be zero and no update would be made.  Under on-policy training, however, $\rho_{t}$ is always one
- Even the simplest combination of bootstrapping and function approximation can be unstable if the updates are not done according to the on-policy distribution
- There are counterexamples that have shown divergence for Q-learning
    - It may be possible to guarantee convergence of Q-learning as long as the behavior policy is sufficiently close to the target policy, for example, when it is the eps-greedy policy
- Another way to try to prevent instability is to use function approximation methods that do not extrapolate from the observed targets.  These methods, called *averagers*, include nearest neighbor methods and locally weighted regression, but not popoular methods such as tile coding and ANNs

# 11.3 The Deadly Triad

- We can summarize the chapter so far by saying that the danger of instability and divergence arises whenever we combine all of the following three elements called the deadly triad:
    - **Function approximation:** A powerful, scalable way of generalizing from a state space much larger than the memory and computational resources (e.g. linear function approximation or ANNs
    - **Bootstrapping:** Update targets that include existing estimates (as in dynamic programming and TD metods) rather than erlying exclusively on actual rewards and complete returns (as in MC methods)
    - **Off-policy training:** Training on a distribution of transitions other than that produced by the target policy.  Sweeping through the state space and updating all states uniformly, as in DP, does not respect the target policy and is an example of off-policy training
- Note that the danger is *not* due to control or to GPI but rather the instability arises in the simpler prediction case whenever it includes all three elements of the deadly triad.
- We need all three to perform certain tasks but if you can get away with using an on-policy method it may be better because it guarantees stability